# Boosting
부스팅(Boosting)이란 단순하고 약한 학습기(Weak Learner)들를 결합해서 보다 정확하고 강력한 학습기(Strong Learner)를 만드는 방식.  
정확도가 낮은 하나의 모델을 만들어 학습 시킨뒤, 그 모델의 예측 오류는 두 번째 모델이 보완한다. 이 두 모델을 합치면 처음보다는 정확한 모델이 만들어 진다. 합쳐진 모델의 예측 오류는 다음 모델에서 보완하여 계속 더하는 과정을 반복한다.

- 약한 학습기들은 앞 학습기가 만든 오류를 줄이는 방향으로 학습한다.
- gradient boosting
    - 처음 모델은 y를 예측 두번째 부터는 앞 모델이 만든 오류를 예측 그것을 앞 모델에 업데이트하면 오류를 줄일 수 있다.
    - 그 오류를 update할 때 뺄까 더할까를 gradient descent 방법을 쓴다. 미분해서 나오는 값의 음수를 취해서 적용. 
    - 학습률을 작게하면 update가 조금씩 크면 많이 하게 된다. 그래서 크게하면 학습데이터에 너무 맞아 과대적합 될 수 있다.

## GradientBoosting
- 개별 모델로 Decision Tree 를 사용한다. 
- depth가 깊지 않은 트리를 많이 연결해서 이전 트리의 오차를 보정해 나가는 방식으로 실행한다.
- 오차를 보정할 때 경사하강법(Gradient descent)을 사용한다.
- 얕은 트리를 많이 연결하여 각각의 트리가 데이터의 일부에 대해 예측을 잘 수행하도록 하고 그런 트리들이 모여 전체 성능을 높이는 것이 기본 아이디어.
- 분류와 회귀 둘다 지원하는 모델 (GradientBoostingClassification, GrandientBoostingRegressor)
- 훈련시간이 많이 걸리고, 트리기반 모델의 특성상 희소한 고차원 데이터에서는 성능이 않좋은 단점이 있다.

### 주요 파라미터
- Decision Tree 의 가지치기 관련 매개변수
    - 각각의 tree가 복잡한 모델이 되지 않도록 한다. 
- learning rate
    - 이전 tree의 오차를 얼마나 강하게 보정할 것인지 제어하는 값. 
    - 값이 크면 보정을 강하게 하여 복잡한 모델을 만든다. 학습데이터의 정확도는 올라가지만 과대적합이 날 수있다. 
    - 값을 작게 잡으면 보정을 약하게 하여 모델의 복잡도를 줄인다. 과대적합을 줄일 수 있지만 성능 자체가 낮아질 수있다.
    - 기본값 : 0.1
- n_estimators
    - tree의 개수 지정. 많을 수록 복잡한 모델이 된다.
- n_iter_no_change, validation_fraction
    - validation_fraction에 지정한 비율만큼 n_iter_no_change에 지정한 반복 횟수동안 검증점수가 좋아 지지 않으면 훈련을 조기 종료한다.

- 보통 max_depth를 낮춰 개별 트리의 복잡도를 낮춘다. (5가 넘지 않게) 그리고 n_estimators를 가용시간, 메모리 한도에 맞춘뒤 적절한 learning_rate을 찾는다.




In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [3]:
data = load_breast_cancer()
X, y = data['data'], data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
y_train.shape, y_test.shape

((426,), (143,))

In [5]:
gb = GradientBoostingClassifier(random_state=1)
gb.fit(X_train, y_train)

GradientBoostingClassifier(random_state=1)

In [6]:
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

In [7]:
accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test)

(1.0, 0.958041958041958)

In [8]:
# feature 중요도
import pandas as pd
fi = gb.feature_importances_
fi_s = pd.Series(fi, index=data['feature_names'])

In [9]:
fi_s.sort_values(ascending=False)

worst radius               0.383871
worst concave points       0.285990
worst perimeter            0.130654
mean concave points        0.046544
worst area                 0.042472
worst texture              0.041187
worst concavity            0.012790
area error                 0.010906
mean texture               0.009120
mean concavity             0.007964
radius error               0.004788
concavity error            0.003404
worst fractal dimension    0.002838
worst symmetry             0.002546
mean area                  0.002540
fractal dimension error    0.002532
mean compactness           0.001572
compactness error          0.001384
mean perimeter             0.001381
smoothness error           0.001340
symmetry error             0.001267
perimeter error            0.000813
mean radius                0.000656
mean fractal dimension     0.000625
texture error              0.000575
worst compactness          0.000097
worst smoothness           0.000066
mean symmetry              0

### GridSearchCV 이용해 최적의 하이퍼파라미터 찾기

In [10]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
param = {
    'n_estimators':[100,200,300,400,500], # tree개수(dafault)
    'learning_rate':[0.001,0.01,0.1], # 학습률
    'max_depth':range(1,5),
    'subsample':[0.5,0.7,1], # 학습시킬 sample의 비율
}

gb = GradientBoostingClassifier(random_state=1)

gs = GridSearchCV(gb,
                 param_grid=param,
                 cv=3,
                 scoring='accuracy',
                 n_jobs=2)

In [11]:
gs.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=1),
             n_jobs=2,
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': range(1, 5),
                         'n_estimators': [100, 200, 300, 400, 500],
                         'subsample': [0.5, 0.7, 1]},
             scoring='accuracy')

In [14]:
gs.cv_results_

{'mean_fit_time': array([0.0528667 , 0.06234638, 0.07263168, 0.10459328, 0.12359436,
        0.14148116, 0.15565157, 0.1844782 , 0.21130149, 0.22048426,
        0.26485912, 0.29168185, 0.26217548, 0.30464435, 0.36251688,
        0.08467237, 0.09745534, 0.1175979 , 0.15388838, 0.19353644,
        0.2357498 , 0.22575482, 0.29557411, 0.36128283, 0.31714296,
        0.38812351, 0.49388576, 0.38210003, 0.48848971, 0.59628344,
        0.10309855, 0.13107212, 0.16149028, 0.21066101, 0.26272527,
        0.31899476, 0.30840262, 0.3960561 , 0.50631261, 0.42531562,
        0.52637378, 0.70412302, 0.54770764, 0.68476033, 0.84187969,
        0.13351043, 0.18686597, 0.21489811, 0.26454528, 0.33558559,
        0.40089973, 0.39072339, 0.50183503, 0.627129  , 0.52904495,
        0.68969154, 0.80880698, 0.66335265, 0.86460861, 1.06701907,
        0.05283825, 0.06156484, 0.07399281, 0.10279854, 0.12396193,
        0.14248967, 0.16617497, 0.18526912, 0.21408351, 0.22118815,
        0.26850351, 0.28930815,

In [ ]:
pred_test = gs.predict(X_test)

In [21]:
gs.best_params_

{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 300, 'subsample': 1}

In [22]:
model = gs.best_estimator_
model.feature_importances_

array([0.00000000e+00, 8.65803996e-03, 1.02445423e-03, 4.71192918e-03,
       7.56521751e-04, 6.69281750e-04, 1.43741151e-02, 1.12214244e-01,
       4.58008512e-05, 1.02227966e-03, 3.72550544e-03, 2.33289667e-03,
       5.07636464e-08, 1.59976825e-02, 2.45791089e-04, 9.09423897e-04,
       7.82762406e-05, 3.69521728e-05, 4.27091800e-04, 5.18519744e-03,
       2.79913093e-01, 2.96846699e-02, 9.73630056e-02, 6.13601810e-02,
       4.73295020e-04, 1.56392178e-03, 2.02925214e-02, 3.35375358e-01,
       1.54926340e-03, 9.15643250e-06])

# XGBoost(Extra Gradient Boost)
- https://xgboost.readthedocs.io/
- Gradient Boost 알고리즘을 기반으로 개선해서 나온 모델.
- 캐글 경진대회에서 상위에 입상한 데이터 과학자들이 사용한 것으로 알려저 유명해짐.
- Gradient Boost의 단점인 느린수행시간을 해결하고 과적합을 제어할 수 있는 규제를 제공하여 성능을 높임.
- 두가지 개발 방법
    - [Scikit-learn 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)
    - [파이썬 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training)
- 설치   
``
pip install xgboost
conda install -y -c anaconda py-xgboost
``



In [1]:
!pip install xgboost

## Scikit-learn 래퍼 XGBoost
- XGBoost를 Scikit-learn프레임워크와 연동할 수 있도록 개발됨.
- Scikit-learn의 Estimator들과 동일한 패턴으로 코드를 작성할 수 있다.
- GridSearchCV나 Pipeline 등 Scikit-learn이 제공하는 다양한 유틸리티들을 사용할 수 있다.
- XGBClassifier: 분류
- XGBRegressor : 회귀 

### 주요 매개변수
- learning_rate : 학습률, 보통 0.01 ~ 0.2 사이의 값 사용
- n_estimators : week tree 개수
- max_depth: 트리의 depth 지정.

In [15]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=200,
                   learning_rate=0.5,
                   max_depth=2,
                   random_state=1)

In [17]:
xgb.fit(X_train, y_train)

[11:32:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.5, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=12, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
pred_train = xgb.predict(X_train)
pred_test = xgb.predict(X_test)

In [19]:
accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test)

(1.0, 0.965034965034965)

In [20]:
fi = xgb.feature_importances_
fi

array([0.0000000e+00, 1.7377844e-02, 0.0000000e+00, 2.7057502e-02,
       9.4335829e-04, 5.7103140e-03, 3.3941608e-02, 9.9188931e-02,
       4.1985163e-03, 2.1932845e-03, 0.0000000e+00, 3.3285678e-04,
       1.7986957e-02, 1.1520459e-02, 0.0000000e+00, 2.2246058e-03,
       0.0000000e+00, 1.2838596e-03, 9.7346597e-04, 8.4046111e-04,
       2.2470552e-01, 1.2404799e-02, 1.2866795e-01, 4.5809176e-02,
       4.1861958e-03, 1.6558268e-04, 1.2857459e-02, 3.3936533e-01,
       4.8814113e-03, 1.1825919e-03], dtype=float32)